In [6]:
import load
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats

class Meta():
    pass

def radial_profile_cut(meta, xx, yy, mm, vx, vy, vz,
                       den_lim=1e6, den_lim2=5e6,
                       mag_lim=25, nbins=100, rmax=50, dr=0.5):
    # 2D photometry. (if rotated towards +y, then use x and z)
    # now assuming +z alignment. 
    rr = np.sqrt(np.square(xx) + np.square(yy))# in kpc unit

    # Account for weights.
    i_sort = np.argsort(rr)
    r_sorted = rr[i_sort]
    m_sorted = mm[i_sort]
 
    rmax = np.max(rr)
    nbins = int(rmax/dr)

    frequency, bins = np.histogram(r_sorted, bins = nbins, range=[0,rmax])
    bin_centers = bins[:-1] + 0.5 * dr # remove the rightmost boundary.
 
    m_radial = np.zeros(nbins)
    ibins = np.concatenate((np.zeros(1), np.cumsum(frequency)))
    for i in range(nbins):
        m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])
        if (m_radial[i]/(2 * np.pi * bin_centers[i] * dr)) < den_lim:
            i_r_cut1 = i-1
            break
 
    i_r_cut2= np.argmax(m_radial/(2 * np.pi * bin_centers * dr) < den_lim2)

    mtot2 = sum(m_radial[:i_r_cut2])
    mtot1 = sum(m_radial[:i_r_cut1])
    i_reff2 = np.argmax(np.cumsum(m_sorted) > (0.5*mtot2))
    i_reff1 = np.argmax(np.cumsum(m_sorted) > (0.5*mtot1))
    meta.reff2 = r_sorted[i_reff2]
    meta.reff  = r_sorted[i_reff1]
    meta.rgal2 = max([bin_centers[i_r_cut2],4*meta.reff2])
    meta.rgal  = max([bin_centers[i_r_cut1],4*meta.reff])#bin_centers[i_r_cut1]

    # velocity center
    # It is not wrong for BCGs to have very large Reff(~50kpc). 
    # But referring the average velocity of stellar particles inside 50kpc 
    # as the system velocity is WRONG.
    # If 1Reff is huge, try smaller aperture when measuring the system velocity.

    i_close = i_sort[:np.argmax(np.cumsum(m_sorted) > (0.1*mtot2))] # 10% closest particles
    meta.vxc = np.average(vx[i_close])
    meta.vyc = np.average(vy[i_close])
    meta.vzc = np.average(vz[i_close])


def rp(gal, meta):
    radial_profile_cut(meta, gal.data['pos'][:,0]
                           , gal.data['pos'][:,1]
                           , gal.data['pos'][:,2]
                           , gal.data['vel'][:,0]
                           , gal.data['vel'][:,1]
                           , gal.data['vel'][:,2])

In [57]:
read=False
nout = 187
wdir = './29176/'

if read:
    sig3d=[]
    mass=[]
    reff=[]
    import tree.halomodule as hmo
    gg = hmo.Halo(base=wdir, nout=nout, is_gal=True, load=True)
    ngal = len(gg.data)
    print("Total {} galaxies in {}".format(ngal, wdir))
    for igal in range(1,ngal + 1):
        gal = load.rd_GM.rd_gal(nout, igal, base=wdir)
        sig3d.append((np.std(gal.data['vel'][:,0])+
                      np.std(gal.data['vel'][:,0])+
                      np.std(gal.data['vel'][:,0]))/np.sqrt(3))
        mass.append(gal.header['mgal'])
 
        meta = Meta()
        rp(gal, meta)
        reff.append(meta.reff)

    mass = np.array(mass)
    sig3d = np.array(sig3d)
    reff = np.array(reff)
    pickle.dump((mass, sig3d, reff),open("mass_sig_r.pickle", 'wb'))
else:
    mass, sig3d, reff = pickle.load(open("mass_sig_r.pickle", 'rb'))


G = 6.67408e-11 #[m3 kg-1 s-2]
msun = 1.989e30 #[kg]
kpc_to_m = 3.0857e16 #[m]

In [21]:
def kernel_density(xx, yy, xmin=None, xmax=None,
                   ymin=None, ymax=None, xpix=100, ypix=100):
    import numpy as np
    import scipy.stats as st
    if xmin is None: xmin = min(xx)
    if xmax is None: xmax = max(xx)
    if ymin is None: ymin = min(yy)
    if ymax is None: ymax = max(yy)

    xpos, ypos = np.mgrid[xmin:xmax:xpix * 1j, ymin:ymax:ypix*1j]
    positions = np.vstack([xpos.ravel(), ypos.ravel()])
    values = np.vstack([xx, yy])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xpos.shape)
    return xx, yy, f

In [71]:
fig, ax = plt.subplots()

# Roughly, G*M = R*sig^2. (virialized)
#sig = sig3d * 1e3 # [m/s]
#mass = mass * msun
#reff = reff * kpc_to_m

xx = mass
yy = sig3d**2 * reff

log = True
clip = True

if log:
    xx = np.log10(xx)
    yy = np.log10(yy)

ax.scatter(xx, yy)
#ax.set_xscale('log')
#ax.set_yscale('log')

def fit_median_subsample(xx, yy, n_per_bin=None, nbins=None):
    if nbins is None:
        if n_per_bin is None:
            return False

    if n_per_bin is None:
        n_per_bin = np.ceil(len(xx)/nbins).astype(int)

    nbins = np.ceil(len(xx)/n_per_bin).astype(int)
    x_bins = np.zeros(nbins)
    y_medians = np.zeros(nbins)

    if clip:
        i_sorted = np.argsort(xx)    
        for i in range(nbins):
            x_bins[i] = np.median(xx[i_sorted[i*n_per_bin:min([len(xx)-1,(i+1)*n_per_bin])]])
            y_medians[i] = np.median(yy[i_sorted[i*n_per_bin:min([len(xx)-1,(i+1)*n_per_bin])]])
    
    return x_bins, y_medians

# Color high density points
x_bins, y_medians = fit_median_subsample(xx, yy, n_per_bin=None, nbins=10)
ax.scatter(x_bins, y_medians, color='r')
slope, intercept, r_value, p_value, std_err = stats.linregress(x_bins,y_medians)
xpoints = np.linspace(min(xx), max(xx), 10)
ax.plot(xpoints, slope * xpoints + intercept, 'r--')

plt.show()

In [67]:
slope * xx[i_bcg] + intercept

60331288.702876493

In [84]:
i_bcg = np.argmax(mass)
print("original Reff of the BCG = ", reff[i_bcg])
print("Projected Reff of the BCG = ", 10**(slope * xx[i_bcg] + intercept) / sig3d[i_bcg]**2)

original Reff of the BCG =  83.3977362433
Projected Reff of the BCG =  12.8648881417


In [49]:
y_medians

29.117063621057895

Now, apply this to derive BCG Reff. (or use as upper limit for any other galaxies)